<a href="https://colab.research.google.com/github/unicamp-dl/IA025_2022S1/blob/main/ex07/aula_07_Beatriz_Celante.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
nome = "Beatriz Celante Vicente"
print(f'Meu nome é {nome}')

Meu nome é Beatriz Celante Vicente


#  Exercício: Modelo de Linguagem (Bengio 2003) - MLP + Embeddings

Neste exercício iremos treinar uma rede neural simples para prever a proxima palavra de um texto, data as palavras anteriores como entrada. Esta tarefa é chamada de "Modelagem da Língua".

Este dataset já possui um tamanho razoável e é bem provável que você vai precisar rodar seus experimentos com GPU.

Alguns conselhos úteis:
- **ATENÇÃO:** o dataset é bem grande. Não dê comando de imprimí-lo.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

In [2]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!pip install transformers

     |████████████████████████████████| 4.2 MB 5.3 MB/s 
     |████████████████████████████████| 84 kB 1.3 MB/s 
     |████████████████████████████████| 596 kB 35.6 MB/s 
     |████████████████████████████████| 6.6 MB 32.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Importação dos pacotes

In [3]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook


In [4]:
# Check which GPU we are using
!nvidia-smi

Wed May 18 13:32:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Implementação do MyDataset

In [6]:
from typing import List


def tokenize(text: str, tokenizer):
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


class MyDataset():
    def __init__(self, texts: List[str], tokenizer, context_size: int):
        self.tokenizer = tokenizer
        self.context_size = context_size
        self.to_id = []

        for text in texts:
          #Faz a tokenização da frase
          text = tokenize(text, self.tokenizer)
          for j in range(0, len(text) - self.context_size):
            x = torch.tensor(text[j: j + self.context_size])
            y = torch.tensor(text[j + self.context_size])
            self.to_id.append((x,y))

    def __len__(self):
        return len(self.to_id)

    def __getitem__(self, idx):
        return self.to_id[idx]

## Teste se sua implementação do MyDataset está correta

In [7]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, context_size=3)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
assert len(dummy_dataset) == 5
print('passou no assert de tamanho do dataset')

first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor(
    [[ 3396, 10303,   125],
     [ 1660,  5971,   785],
     [ 5971,   785,   125],
     [  785,   125,  1847],
     [  125,  1847, 13779]])

correct_first_batch_target = torch.LongTensor([13239,   125,  1847, 13779, 15616])

assert torch.equal(first_batch_input, correct_first_batch_input)
print('Passou no assert de input')
assert torch.equal(first_batch_target, correct_first_batch_target)
print('Passou no assert de target')

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

passou no assert de tamanho do dataset
Passou no assert de input
Passou no assert de target


# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [8]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula7/sample_brwac.txt

--2022-05-18 13:32:18--  https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula7/sample_brwac.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.201.128, 64.233.183.128, 142.250.148.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.201.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 123983611 (118M) [text/plain]
Saving to: ‘sample_brwac.txt’

sample_brwac.txt    100%[===================>] 118.24M   121MB/s    in 1.0s    

2022-05-18 13:32:19 (121 MB/s) - ‘sample_brwac.txt’ saved [123983611/123983611]



In [9]:
# Load datasets
context_size = 9

valid_examples = 100
test_examples = 100
texts = open('sample_brwac.txt').readlines()

print('Truncating for debugging purposes.')
texts = texts[:500]  

training_texts = texts[:-(valid_examples + test_examples)]
valid_texts = texts[-(valid_examples + test_examples):-test_examples]
test_texts = texts[-test_examples:]

training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, context_size=context_size)
valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, context_size=context_size)
test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, context_size=context_size)

Truncating for debugging purposes.


In [10]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 406905
valid examples: 135562
test examples: 136690


A implementação abaixo foi baseada no artigo https://abhinavcreed13.github.io/blog/bengio-trigram-nplm-using-pytorch/

In [11]:
import torch.nn.functional as F

class LanguageModel(torch.nn.Module):

    def __init__(self, vocab_size, context_size, embedding_dim, hidden_size):
      """
      Implements the Neural Language Model proposed by Bengio et al."

      Args:
          vocab_size (int): Size of the input vocabulary.
          context_size (int): Size of the sequence to consider as context for prediction.
          embedding_dim (int): Dimension of the embedding layer for each word in the context.
          hidden_size (int): Size of the hidden layer.
      """
      super(LanguageModel, self).__init__()
      self.vocab_size = vocab_size
      self.context_size = context_size
      self.embedding_dim = embedding_dim
      self.hidden_size = hidden_size

      self.embeddings = nn.Embedding(self.vocab_size, self.embedding_dim)
      self.linear1 = nn.Linear(self.context_size * self.embedding_dim, self.hidden_size)
      self.linear2 = nn.Linear(self.hidden_size, vocab_size, bias = False)

    def forward(self, inputs):
      """
      Args: inputs is a LongTensor of shape (batch_size, context_size)
      """
      # Concatena os embeddings
      embeds = self.embeddings(inputs).view((-1,self.context_size * self.embedding_dim))
      #Primeira camada linear
      out = self.linear1(embeds)
      # Tangente hiperbólica
      out = torch.tanh(out)
      # Segunda camada linear
      out = self.linear2(out)
      # Log das probabilidades com Softmax
      log_probs = F.log_softmax(out, dim=1)

      return log_probs

## Teste o modelo com um exemplo

In [12]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=64,
    hidden_size=128,
).to(device)

sample_train, _ = next(iter(DataLoader(training_dataset)))
sample_train_gpu = sample_train.to(device)
model(sample_train_gpu).shape

torch.Size([1, 29794])

In [13]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 5794304


## Assert da Perplexidade


In [14]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)


def perplexity(logits, target):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, vocab_size)
        target: a LongTensor of shape (batch_size,)

    Returns:
        A float corresponding to the perplexity.
    """
    loss = torch.nn.CrossEntropyLoss()(logits, target)
    perplexity =  torch.exp(loss).sum()

    return perplexity


n_examples = 1000

sample_train, target_token_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
sample_train_gpu = sample_train.to(device)
target_token_ids = target_token_ids.to(device)
logits = model(sample_train_gpu)

my_perplexity = perplexity(logits=logits, target=target_token_ids)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=2000)
print('Passou o no assert da perplexidade')

my perplexity:              31288
correct initial perplexity: 29794
Passou o no assert da perplexidade


## Laço de Treinamento e Validação

In [18]:
max_examples = 10_000_000
eval_every_steps = 5000
lr = 3e-5


model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=128,
    hidden_size=256,
).to(device)

train_loader = DataLoader(training_dataset, batch_size=64, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=64)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input, target):
    model.train()
    model.zero_grad()

    logits = model(input.to(device))
    loss = nn.functional.cross_entropy(logits, target.to(device))
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input, target):
    logits = model(input)
    loss = nn.functional.cross_entropy(logits, target)
    return loss.item()


train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for input, target in train_loader:
        loss = train_step(input.to(device), target.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(input.to(device), target.to(device))
                    for input, target in validation_loader]))

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(input)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

0 steps; 0 examples so far; train ppl: 29988.89, valid ppl: 31016.80
5000 steps; 320000 examples so far; train ppl: 3985.87, valid ppl: 2334.67
10000 steps; 640000 examples so far; train ppl: 1573.21, valid ppl: 1816.59
15000 steps; 960000 examples so far; train ppl: 1248.25, valid ppl: 1614.90
20000 steps; 1280000 examples so far; train ppl: 1065.68, valid ppl: 1490.53
25000 steps; 1600000 examples so far; train ppl: 934.15, valid ppl: 1397.97
30000 steps; 1920000 examples so far; train ppl: 820.98, valid ppl: 1328.86
35000 steps; 2240000 examples so far; train ppl: 746.63, valid ppl: 1273.06
40000 steps; 2560000 examples so far; train ppl: 671.47, valid ppl: 1225.98
45000 steps; 2880000 examples so far; train ppl: 622.29, valid ppl: 1186.68
50000 steps; 3200000 examples so far; train ppl: 567.35, valid ppl: 1156.19
55000 steps; 3520000 examples so far; train ppl: 524.25, valid ppl: 1129.25
60000 steps; 3840000 examples so far; train ppl: 488.26, valid ppl: 1101.95
65000 steps; 416000

## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [19]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(input.to(device), target.to(device))
        for input, target in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 980.6716906695373


## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [20]:
prompt = "Todas as cartas de amor são ridículas. Não seriam"  # Ex: 'Eu gosto de comer pizza pois me faz'
max_output_tokens = 10

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Todas as cartas de amor são ridículas. Não seriam a
Todas as cartas de amor são ridículas. Não seriam a primeira
Todas as cartas de amor são ridículas. Não seriam a primeira vez
Todas as cartas de amor são ridículas. Não seriam a primeira vez que
Todas as cartas de amor são ridículas. Não seriam a primeira vez que o
Todas as cartas de amor são ridículas. Não seriam a primeira vez que o governo
Todas as cartas de amor são ridículas. Não seriam a primeira vez que o governo.
Todas as cartas de amor são ridículas. Não seriam a primeira vez que o governo. "
Todas as cartas de amor são ridículas. Não seriam a primeira vez que o governo. ".
Todas as cartas de amor são ridículas. Não seriam a primeira vez que o governo. ". O
